# Cell states of the tumor immune microenvironment (TIME)
author: Margaret Paiva

In [16]:
import numpy as np
import pandas as pd
import scanpy as sc
from scipy.stats import mannwhitneyu
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
# # table with clinical responses
# # complete response (CR) and partial response (PR) for responders, 
# # stable disease (SD) and progressive disease (PD) for nonresponders
# meta = pd.read_csv('data/GSE120575_patient_ID_single_cells.txt.gz', 
#                    sep="\t", encoding="latin", 
#                    skiprows=19).iloc[:, :7]
# # remove additional information after sample table
# meta = meta.iloc[:16291]

# # rename columns by removing the `characteristics: ` prefix to make it more concise
# meta.columns = [x.replace("characteristics: ", "") for x in meta.columns]

# # rename the long `patient ID (...)` column to simple sample_id
# meta.rename(
#     columns={"patinet ID (Pre=baseline; Post= on treatment)": "sample_id"},
#     inplace=True
# )

# # add columns that seperate sample_id for later analysis
# meta["patient_id"] = [x.split("_")[1] for x in meta.sample_id]
# meta["time_point"] = [x.split("_")[0] for x in meta.sample_id]

# meta.head(3)

,Sample name,title,source name,organism,sample_id,response,therapy,patient_id,time_point
0,Sample 1,A10_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
1,Sample 2,A11_P1_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
2,Sample 3,A11_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre


In [35]:
meta.head(50)

,Sample name,title,source name,organism,sample_id,response,therapy,patient_id,time_point
0,Sample 1,A10_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
1,Sample 2,A11_P1_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
2,Sample 3,A11_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
3,Sample 4,A11_P4_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
4,Sample 5,A12_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
5,Sample 6,A12_P6_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
6,Sample 7,A2_P1_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
7,Sample 8,A2_P4_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
8,Sample 9,A3_P1_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre
9,Sample 10,A3_P3_M11,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre


In [7]:
ge = sc.read_h5ad('data/GSE120575_annotated.h5ad')
ge

AnnData object with n_obs × n_vars = 16215 × 45884
    obs: 'Sample name', 'source name', 'organism', 'sample_id', 'response', 'therapy', 'patient_id', 'time_point', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'cell_type'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'cell_type_colors', 'neighbors', 'pca', 'rank_genes_groups', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [46]:
ge.obs.head(50)

,Sample name,source name,organism,sample_id,response,therapy,patient_id,time_point,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,cell_type
title,,,,,,,,,,,,,,
A10_P3_M11,Sample 1,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,2050,2050,14624.809570,171.209991,1.170682,Dendritic cells
A11_P1_M11,Sample 2,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,1573,1573,11933.599609,160.350006,1.343685,B cells
A11_P3_M11,Sample 3,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,1591,1590,11877.649414,176.229980,1.483711,Memory T cells
A11_P4_M11,Sample 4,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,2909,2909,19693.539062,146.829987,0.745574,Regulatory T cells
A12_P3_M11,Sample 5,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,1211,1210,9182.320312,196.079987,2.135408,others
A12_P6_M11,Sample 6,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,2715,2715,18240.398438,140.460007,0.770049,CD8+ T cells
A2_P1_M11,Sample 7,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,1139,1139,8914.789062,158.120010,1.773682,others
A2_P4_M11,Sample 8,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,2247,2246,15485.159180,160.409988,1.035895,CD8+ T cells
A3_P1_M11,Sample 9,Melanoma single cell,Homo sapiens,Pre_P1,Responder,anti-CTLA4,P1,Pre,2881,2881,19709.910156,156.699982,0.795031,others


In [43]:
ge.obs[(ge.obs.sample_id=='Post_P1') & (ge.obs.time_point=='Pre')]

,Sample name,source name,organism,sample_id,response,therapy,patient_id,time_point,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,cell_type
title,,,,,,,,,,,,,,


In [58]:
n = 1
for g in ["sample_id",
     "time_point", 
     "response",
     "cell_type"]:
    n *= data[g].nunique()
print(n)'

1728


In [ ]:
"23" -> 2 + 3 + 23
# f("342") -> 3 + 4 + 2 + 34 + 42 + 342
# WRONG:
# def f(s):
#    return sum([(int(c) for c in s)] + int(s))
def f(s):
    n = 0
    for i in range(len(s)):
        for j in range(len(s)-i):
            n += int(s[j:j+i+1])
    return n

In [53]:
data = pd.DataFrame(ge.obs)

# group cells by cluster and sample_id
cluster_pct = data.groupby(
    ["sample_id",
     "time_point", 
     "response",
     "cell_type"],
    observed=True
).size()
# .reset_index().rename(
#     columns={0: "cells"}
# )

# # calculate percentage of each cluster in each sample
# cluster_pct = cluster_pct.merge(
#     data.groupby("sample_id").size().reset_index()
# ).rename(columns={0: "total_cells"})

# cluster_pct["percent"] = cluster_pct.cells / cluster_pct.total_cells
cluster_pct[:40]

sample_id  time_point  response       cell_type         
Pre_P1     Pre         Responder      Dendritic cells        17
                                      B cells                 4
                                      Memory T cells          9
                                      Regulatory T cells     19
                                      others                 23
                                      CD8+ T cells          135
                                      CD4+ T cells           10
                                      Macrophages             4
                                      Exhausted T cells       8
Post_P1    Post        Responder      Dendritic cells        33
                                      B cells                10
                                      Memory T cells         41
                                      Regulatory T cells     33
                                      others                113
                                      CD4+ T ce

In [45]:
cluster_pct.head(50)

,sample_id,time_point,response,cell_type,cells
0,Post_P1,Post,Non-responder,B cells,0
1,Post_P1,Post,Non-responder,CD4+ T cells,0
2,Post_P1,Post,Non-responder,CD8+ T cells,0
3,Post_P1,Post,Non-responder,Dendritic cells,0
4,Post_P1,Post,Non-responder,Exhausted T cells,0
5,Post_P1,Post,Non-responder,Macrophages,0
6,Post_P1,Post,Non-responder,Memory T cells,0
7,Post_P1,Post,Non-responder,Regulatory T cells,0
8,Post_P1,Post,Non-responder,others,0
9,Post_P1,Post,Responder,B cells,10


In [15]:
pct_mat = cluster_pct.pivot_table(
    index=["time_point", "patient_id", "sample_id", "response"],
    columns=["cell_type"], values="percent",
    fill_value=0
)

cell_type                                       B cells  CD4+ T cells  \
time_point patient_id sample_id response                                
Post       P1         Post_P1   Non-responder  0.000000      0.000000   
                                Responder      0.034602      0.041522   
                      Post_P1_2 Non-responder  0.084986      0.096317   
                                Responder      0.000000      0.000000   
                      Post_P2   Non-responder  0.000000      0.000000   
...                                                 ...           ...   
Pre        P35        Pre_P31   Responder      0.000000      0.000000   
                      Pre_P33   Non-responder  0.000000      0.000000   
                                Responder      0.000000      0.000000   
                      Pre_P35   Non-responder  0.000000      0.000000   
                                Responder      0.378151      0.084034   

cell_type                                      CD8+ T cells  Dendritic cells  \
time_point patient_id sample_id response                                       
Post       P1         Post_P1   Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.114187   
                      Post_P1_2 Non-responder      0.645892         0.014164   
                                Responder          0.000000         0.000000   
                      Post_P2   Non-responder      0.000000         0.000000   
...                                                     ...              ...   
Pre        P35        Pre_P31   Responder          0.000000         0.000000   
                      Pre_P33   Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.000000   
                      Pre_P35   Non-responder      0.000000         0.000000   
                                Responder          0.201681         0.042017   

cell_type                                      Exhausted T cells  Macrophages  \
time_point patient_id sample_id response                                        
Post       P1         Post_P1   Non-responder           0.000000     0.000000   
                                Responder               0.131488     0.031142   
                      Post_P1_2 Non-responder           0.002833     0.011331   
                                Responder               0.000000     0.000000   
                      Post_P2   Non-responder           0.000000     0.000000   
...                                                          ...          ...   
Pre        P35        Pre_P31   Responder               0.000000     0.000000   
                      Pre_P33   Non-responder           0.000000     0.000000   
                                Responder               0.000000     0.000000   
                      Pre_P35   Non-responder           0.000000     0.000000   
                                Responder               0.046218     0.012605   

cell_type                                      Memory T cells  \
time_point patient_id sample_id response                        
Post       P1         Post_P1   Non-responder        0.000000   
                                Responder            0.141869   
                      Post_P1_2 Non-responder        0.016997   
                                Responder            0.000000   
                      Post_P2   Non-responder        0.000000   
...                                                       ...   
Pre        P35        Pre_P31   Responder            0.000000   
                      Pre_P33   Non-responder        0.000000   
                                Responder            0.000000   
                      Pre_P35   Non-responder        0.000000   
                                Responder            0.063025   

cell_type                                      Regulatory T cells    others  
time_point patient_id sample_id

In [22]:
pct_mat.head(10)

cell_type                                       B cells  CD4+ T cells  \
time_point patient_id sample_id response                                
Post       P1         Post_P1   Non-responder  0.000000      0.000000   
                                Responder      0.034602      0.041522   
                      Post_P1_2 Non-responder  0.084986      0.096317   
                                Responder      0.000000      0.000000   
                      Post_P2   Non-responder  0.000000      0.000000   
                                Responder      0.000000      0.000000   
                      Post_P3   Non-responder  0.000000      0.000000   
                                Responder      0.000000      0.000000   
                      Post_P3_2 Non-responder  0.000000      0.000000   
                                Responder      0.000000      0.000000   

cell_type                                      CD8+ T cells  Dendritic cells  \
time_point patient_id sample_id response                                       
Post       P1         Post_P1   Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.114187   
                      Post_P1_2 Non-responder      0.645892         0.014164   
                                Responder          0.000000         0.000000   
                      Post_P2   Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.000000   
                      Post_P3   Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.000000   
                      Post_P3_2 Non-responder      0.000000         0.000000   
                                Responder          0.000000         0.000000   

cell_type                                      Exhausted T cells  Macrophages  \
time_point patient_id sample_id response                                        
Post       P1         Post_P1   Non-responder           0.000000     0.000000   
                                Responder               0.131488     0.031142   
                      Post_P1_2 Non-responder           0.002833     0.011331   
                                Responder               0.000000     0.000000   
                      Post_P2   Non-responder           0.000000     0.000000   
                                Responder               0.000000     0.000000   
                      Post_P3   Non-responder           0.000000     0.000000   
                                Responder               0.000000     0.000000   
                      Post_P3_2 Non-responder           0.000000     0.000000   
                                Responder               0.000000     0.000000   

cell_type                                      Memory T cells  \
time_point patient_id sample_id response                        
Post       P1         Post_P1   Non-responder        0.000000   
                                Responder            0.141869   
                      Post_P1_2 Non-responder        0.016997   
                                Responder            0.000000   
                      Post_P2   Non-responder        0.000000   
                                Responder            0.000000   
                      Post_P3   Non-responder        0.000000   
                                Responder            0.000000   
                      Post_P3_2 Non-responder        0.000000   
                                Responder            0.000000   

cell_type                                      Regulatory T cells    others  
time_point patient_id sample_id response                                     
Post       P1         Post_P1   Non-responder            0.000000  0.000000  
                                Responder                0.114187  0.391003  
                      Post_P1_2 Non-responder            0.096317  0.031161  
                  

In [19]:
# Calculate P values using Mann-Whitney U test
def pct_test(data, cluster):
    _data = data[cluster].reset_index()
    resp_pct = _data.loc[_data.response == "Responder", cluster].values
    nonresp_pct = _data.loc[_data.response == "Non-responder", cluster].values
    stat, pval = mannwhitneyu(resp_pct, nonresp_pct)
    return {"cluster": cluster,
            "median_responder_pct": np.median(resp_pct),
            "median_nonresponder_pct": np.median(nonresp_pct),
            "pvalue": pval}

In [20]:
res = pd.DataFrame(
    [pct_test(pct_mat, cluster) for cluster in pct_mat.columns]
)

res.sort_values("pvalue")

,cluster,median_responder_pct,median_nonresponder_pct,pvalue
2,CD8+ T cells,0.0,0.0,0.027898
0,B cells,0.0,0.0,0.029023
3,Dendritic cells,0.0,0.0,0.042338
7,Regulatory T cells,0.0,0.0,0.042425
6,Memory T cells,0.0,0.0,0.042579
1,CD4+ T cells,0.0,0.0,0.042670
8,others,0.0,0.0,0.042694
4,Exhausted T cells,0.0,0.0,0.042927
5,Macrophages,0.0,0.0,0.050771
